In [ ]:
# SISO 5G gNB-UE Simulation using AIRSTRAN D 2200

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

# Other imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mitsuba as mi
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="invalid value encountered in multiply")
warnings.filterwarnings("ignore", category=UserWarning, module="jupyter_client")

# Import relevant components from Sionna RT
from sionna.rt import load_scene, Transmitter, Receiver, Transmitter, Camera, PathSolver
from sionna.rt import AntennaArray, PlanarArray, SceneObject, ITURadioMaterial
from sionna.rt.antenna_pattern import antenna_pattern_registry

# Load empty scene
scene = load_scene("/home/tingjunlab/Development/geo2sigmap/scenes/Duke_Perkins/scene.xml")

In [ ]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

In [ ]:
# Install plyfile if not available
try:
    from plyfile import PlyData
except ImportError:
    import os
    os.system("pip install plyfile")
    from plyfile import PlyData

In [ ]:
# ============================================
# SISO Configuration: gNB to UE
# ============================================

# Set the operating frequency (n48 band for 5G)
scene.frequency = 3.7e9  # 3.7 GHz

# Define UE position (fixed to start)
ue_position = [10.0, 0.0, 0.0]   # UE position (x, y, z in meters)

# ============================================
# Antenna Configuration
# ============================================

# gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
gnb_pattern = gnb_pattern_factory(polarization="V")

# UE antenna: Isotropic pattern (typical for mobile devices)
ue_pattern_factory = antenna_pattern_registry.get("iso")
ue_pattern = ue_pattern_factory(polarization="V")

# SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

# Configure antenna arrays
scene.tx_array = AntennaArray(
    antenna_pattern=gnb_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

scene.rx_array = AntennaArray(
    antenna_pattern=ue_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

# ============================================
# Add Receiver to Scene
# ============================================

# Create UE receiver
rx = Receiver(name="ue", position=ue_position, display_radius=0.03)
scene.add(rx)

# ============================================
# Configure Propagation Environment
# ============================================

# Disable scattering for basic simulation
for radio_material in scene.radio_materials.values():
    radio_material.scattering_coefficient = 0.0

In [ ]:
from scene_parser import extract_building_info
from tx_placement import TxPlacement

In [ ]:
# ============================================
# Place gNB on a Specific Building
# ============================================

scene_xml_path = "/home/tingjunlab/Development/geo2sigmap/scenes/Duke_Perkins/scene.xml"
building_info = extract_building_info(scene_xml_path, verbose=True)

selected_building_id = 5  # Change this to your desired building number

# TxPlacement will create the transmitter if it doesn't exist and place it on the building
# Correct parameter order: (scene, tx_name, scene_xml_path, building_id, offset)
tx_placer = TxPlacement(scene, "gnb", scene_xml_path, selected_building_id, offset=5.0)
tx_placer.set_rooftop_center()

# Get reference to the transmitter (already added to scene by TxPlacement)
tx = tx_placer.tx
# Convert to flat numpy array instead of nested list
gnb_position = tx.position.numpy().flatten().tolist()

# Point antenna toward UE
tx.look_at(ue_position)

print(f"\nSuccess! gNB placed on building {selected_building_id}")
print(f"Position: {gnb_position}")

# ============================================
# Compute Propagation Paths
# ============================================

# Instantiate path solver
p_solver = PathSolver()

# Compute propagation paths
paths = p_solver(
    scene=scene,
    max_depth=5,
    max_num_paths_per_src=5000,
    los=True,
    specular_reflection=True,
    diffuse_reflection=False,
    diffraction=True,
    edge_diffraction=False,
    refraction=False,
    synthetic_array=False,
    seed=41,
)

# ============================================
# Visualize Scene
# ============================================

# Setup camera
cam = Camera(position=(100.0, 100.0, 50.0))
cam.look_at(gnb_position)

# Preview the scene with propagation paths
scene.preview(
    paths=paths,
    resolution=[1000, 1000],
    clip_at=50,
    show_orientations=True
)

In [ ]:
from boresight_pathsolver import (
    optimize_boresight_pathsolver,
    create_zone_mask
)
import matplotlib.pyplot as plt

# ============================================
# NEW BINARY MASK APPROACH
# ============================================
# Instead of creating a target radiomap with specific power levels,
# we create a BINARY MASK that defines the coverage zone.
# The optimizer will maximize coverage within this zone.

# Map configuration
#map_config = {
#    'center': [gnb_position[0], gnb_position[1], 0],
#    'size': [500, 500],
#    'cell_size': (1, 1),
#    'ground_height': 0.0,
#}

# ============================================
# OPTION 1: Angular Sector Zone (RECOMMENDED)
# ============================================
# Create a wedge-shaped sector pointing from the TX
#zone_mask, naive_look_at, zone_stats = create_zone_mask(
#    map_config=map_config,
#    zone_type='angular_sector',
#    origin_point=gnb_position,
#    zone_params={
#        'angle_start': 90,
#        'angle_end': 90+65,      # Try 65° to match TR38.901 beamwidth
#        'radius': 200         # Larger radius
#    },
#    target_height=0.0,
#    scene_xml_path="/home/tingjunlab/Development/geo2sigmap/scenes/Duke_Perkins/scene.xml",
#    exclude_buildings=True           # ← This is True by default
#)

map_config = {
    'center': [gnb_position[0], gnb_position[1], 0],
    'size': [500, 500],
    'cell_size': (1, 1),
    'ground_height': 0.0,
}

zone_params = {
    'center': (-70, 70),
    'width': 100,
    'height': 100
}

zone_mask, naive_look_at, zone_stats = create_zone_mask(
    map_config=map_config,
    zone_type='box',
    zone_params=zone_params,
    target_height=1.5,
    scene_xml_path=scene_xml_path,
    exclude_buildings=True
)

print("=" * 70)
print("ZONE MASK CREATED")
print("=" * 70)
print(f"Zone type: Angular sector (0° to 120°, radius 100m)")
print(f"Zone contains {zone_stats['num_cells']} grid cells")
print(f"Zone coverage: {zone_stats['coverage_fraction']*100:.1f}% of map")
print(f"Naive baseline look-at: {zone_stats['look_at_xyz']}")
print(f"Zone centroid: {zone_stats['centroid_xy']}")
print()

# ============================================
# OPTION 2: Box Zone (Alternative)
# ============================================
# Uncomment to use a rectangular zone instead:
"""
zone_mask, naive_look_at, zone_stats = create_zone_mask(
    map_config=map_config,
    zone_type='box',
    zone_params={
        'center': (gnb_position[0] + 50, gnb_position[1] + 50),
        'width': 80,   # 80m wide
        'height': 60   # 60m tall
    },
    target_height=1.5
)
"""

# ============================================
# Visualize the Zone Mask
# ============================================
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(
    zone_mask,
    origin='lower',
    cmap='RdYlGn',
    extent=[
        map_config['center'][0] - map_config['size'][0]/2,
        map_config['center'][0] + map_config['size'][0]/2,
        map_config['center'][1] - map_config['size'][1]/2,
        map_config['center'][1] + map_config['size'][1]/2,
    ],
    vmin=0,
    vmax=1
)
plt.colorbar(im, ax=ax, label='Zone Mask (1.0 = in zone, 0.0 = out)')
ax.plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
ax.plot(zone_stats['centroid_xy'][0], zone_stats['centroid_xy'][1], 'bo',
        markersize=10, label='Zone Centroid')
ax.plot(zone_stats['look_at_xyz'][0], zone_stats['look_at_xyz'][1], 'g^',
        markersize=10, label='Naive Baseline')
ax.set_title(f'Coverage Zone Mask ({zone_stats["num_cells"]} cells)')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

# ============================================
# Run Optimization with Binary Mask
# ============================================
print("=" * 70)
print("STARTING BINARY MASK OPTIMIZATION")
print("=" * 70)

best_boresight, loss_hist, bore_hist, grad_hist, cov_stats = optimize_boresight_pathsolver(
    scene=scene,
    tx_name="gnb",
    map_config=map_config,
    scene_xml_path=scene_xml_path,
    zone_mask=zone_mask, 
    initial_boresight=naive_look_at.tolist(),  # Use zone center as starting point
    num_sample_points=100,
    learning_rate=3.0,
    num_iterations=50,
    loss_type='coverage_maximize', 
    use_grid_search_init=True,
    # Alternative loss types:
    #power_threshold_dbm=-70,
    # loss_type='percentile_maximize'
    verbose=True,
    seed=33,
    # Enable frame saving
    save_radiomap_frames=False,  # Set to True to save frames
    frame_save_interval=10,
    output_dir="/home/tingjunlab/Development/optimize_tx/optimization_gif"
)

print(f"\nOptimization complete!")
print(f"Final coverage stats: {cov_stats}")

In [ ]:
from boresight_pathsolver import compare_boresight_performance

# After optimization is complete, compare results
fig, comparison_stats = compare_boresight_performance(
    scene=scene,
    tx_name="gnb",
    map_config=map_config,
    zone_mask=zone_mask,
    naive_boresight=naive_look_at,  # From create_zone_mask
    optimized_boresight=best_boresight,  # From optimize_boresight_pathsolver
    title="Boresight Optimization Results"
)

plt.show()

# Print detailed stats
print(f"Mean improvement: {comparison_stats['improvement_mean_dB']:.2f} dB")
print(f"Median improvement: {comparison_stats['improvement_median_dB']:.2f} dB")
print(f"Percentage improvement: {comparison_stats['improvement_percent']:.1f}%")

In [ ]:
from sionna.rt import RadioMapSolver
import matplotlib.pyplot as plt

# Apply the optimized boresight
tx.look_at(mi.Point3f(float(best_boresight[0]), float(best_boresight[1]), float(best_boresight[2])))

print("=" * 70)
print("RADIO MAP VISUALIZATION (OPTIMIZED)")
print("=" * 70)
print(f"Optimized boresight: ({best_boresight[0]:.1f}, {best_boresight[1]:.1f}, {best_boresight[2]:.1f})")
print(f"TX location: {tx.position}")
print(f"Coverage zone: {cov_stats['num_samples_in_zone']}/{cov_stats['num_samples_total']} samples")
print(f"Final loss: {cov_stats['final_loss']:.4f}")
print()

# Generate radio map matching the optimization configuration
rm_solver = RadioMapSolver()
rm = rm_solver(
    scene,
    max_depth=5,
    samples_per_tx=int(6e8),
    cell_size=map_config['cell_size'],
    center=map_config['center'],
    orientation=[0, 0, 0],
    size=map_config['size'],
    los=True,
    specular_reflection=True,
    diffuse_reflection=True,
    refraction=False,
    stop_threshold=None,
)

# Extract signal levels
rss_watts = rm.rss.numpy()[0, :, :]
signal_strength_dBm = 10.0 * np.log10(rss_watts + 1e-30) + 30.0

# ============================================
# Analyze Coverage in the Zone
# ============================================
# Extract power values only in the zone
zone_power = signal_strength_dBm[zone_mask == 1.0]

print("Coverage Analysis:")
print(f"  Mean power in zone: {np.mean(zone_power):.2f} dBm")
print(f"  Median power in zone: {np.median(zone_power):.2f} dBm")
print(f"  Min power in zone: {np.min(zone_power):.2f} dBm")
print(f"  Max power in zone: {np.max(zone_power):.2f} dBm")
print(f"  Std dev in zone: {np.std(zone_power):.2f} dB")
print()

# Count cells above various thresholds
thresholds = [-90, -85, -80, -75, -70]
for thresh in thresholds:
    above = np.sum(zone_power > thresh)
    pct = 100.0 * above / len(zone_power)
    print(f"  Above {thresh} dBm: {above}/{len(zone_power)} ({pct:.1f}%)")
print()

# ============================================
# Visualize Results
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

extent = [
    map_config['center'][0] - map_config['size'][0]/2,
    map_config['center'][0] + map_config['size'][0]/2,
    map_config['center'][1] - map_config['size'][1]/2,
    map_config['center'][1] + map_config['size'][1]/2,
]

# Plot 1: Zone Mask
im1 = axes[0].imshow(zone_mask, origin='lower', cmap='RdYlGn', extent=extent, vmin=0, vmax=1)
axes[0].plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
axes[0].plot(best_boresight[0], best_boresight[1], 'b^', markersize=15, label='Optimized Aim')
axes[0].set_title('Coverage Zone Mask')
axes[0].set_xlabel('X (m)')
axes[0].set_ylabel('Y (m)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
plt.colorbar(im1, ax=axes[0], label='Mask Value')

# Plot 2: RadioMap (Signal Strength)
im2 = axes[1].imshow(signal_strength_dBm, origin='lower', cmap='viridis', 
                     extent=extent, vmin=-120, vmax=-60)
axes[1].plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
axes[1].plot(best_boresight[0], best_boresight[1], 'w^', markersize=15, label='Optimized Aim')
axes[1].set_title(f'Optimized Signal Strength\nMean in zone: {np.mean(zone_power):.1f} dBm')
axes[1].set_xlabel('X (m)')
axes[1].set_ylabel('Y (m)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.colorbar(im2, ax=axes[1], label='Signal Strength (dBm)')

# Plot 3: Masked Coverage (only show zone)
masked_signal = np.where(zone_mask == 1.0, signal_strength_dBm, np.nan)
im3 = axes[2].imshow(masked_signal, origin='lower', cmap='viridis',
                     extent=extent, vmin=-120, vmax=-60)
axes[2].plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
axes[2].plot(best_boresight[0], best_boresight[1], 'w^', markersize=15, label='Optimized Aim')
axes[2].set_title(f'Coverage in Zone Only\nMedian: {np.median(zone_power):.1f} dBm')
axes[2].set_xlabel('X (m)')
axes[2].set_ylabel('Y (m)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
plt.colorbar(im3, ax=axes[2], label='Signal Strength (dBm)')

plt.tight_layout()
plt.show()

# ============================================
# Optimization Diagnostics
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss history
axes[0].plot(loss_hist, 'b-', linewidth=2)
axes[0].set_title('Loss History')
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss')
axes[0].grid(True, alpha=0.3)

# Gradient norms
axes[1].plot(grad_hist, 'r-', linewidth=2)
axes[1].set_title('Gradient Norm History')
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Gradient Norm')
axes[1].set_yscale('log')
axes[1].grid(True, alpha=0.3)

# Boresight trajectory (XY plane)
bore_arr = np.array(bore_hist)
axes[2].plot(bore_arr[:, 0], bore_arr[:, 1], 'g-', alpha=0.5, label='Trajectory')
axes[2].plot(bore_arr[0, 0], bore_arr[0, 1], 'ro', markersize=10, label='Start')
axes[2].plot(bore_arr[-1, 0], bore_arr[-1, 1], 'b^', markersize=12, label='Final')
axes[2].plot(gnb_position[0], gnb_position[1], 'r*', markersize=15, label='TX')
axes[2].set_title('Boresight Optimization Trajectory (XY)')
axes[2].set_xlabel('X (m)')
axes[2].set_ylabel('Y (m)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].axis('equal')

plt.tight_layout()
plt.show()

# Scene preview with optimized radiomap
scene.preview(radio_map=rm)